In [9]:
import os, sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

# Dataset, dataloader
import torch.utils.data as Data
from torchvision import transforms

# Model
import torch.nn as nn
import torch.nn.functional as F

# Training
from torch.autograd import Variable

# Progress bar
from tqdm import tqdm

# Save file
from datetime import date

# Confusion Matrix
import sklearn.metrics as skm

import argparse
import collections

# import from parent dir
sys.path.insert(1, os.path.realpath(os.path.pardir))    #sys.path.append('../')
from utils import read_json, ROOT_DIR
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser

import warnings
warnings.filterwarnings('ignore')

In [7]:
par_dir = os.path.realpath(os.path.pardir)
cur_dir = os.getcwd()
csv_dir = os.path.join(ROOT_DIR, 'data', 'train_pivot.csv')

print("Parent dir: {}".format(par_dir))
print("Current dir: {}".format(cur_dir))
print('Root dir: {}'.format(ROOT_DIR))
print('Img dir: {}'.format(csv_dir))

Parent dir: D:\jupyter-notebook\Kaggle\Steel-Defect-Detection-Pytorch
Current dir: d:\jupyter-notebook\Kaggle\Steel-Defect-Detection-Pytorch\notebook
Root dir: D:\jupyter-notebook\Kaggle\Steel-Defect-Detection-Pytorch\utils\..
Img dir: D:\jupyter-notebook\Kaggle\Steel-Defect-Detection-Pytorch\utils\..\data\train_pivot.csv


In [8]:
pd.read_csv(csv_dir)

,ImageId,1,2,3,4
0,0002cc93b.jpg,True,False,False,False
1,0007a71bf.jpg,False,False,True,False
2,000a4bcdd.jpg,True,False,False,False
3,000f6bf48.jpg,False,False,False,True
4,0014fce06.jpg,False,False,True,False
...,...,...,...,...,...
6661,ffcf72ecf.jpg,False,False,True,False
6662,fff02e9c5.jpg,False,False,True,False
6663,fffe98443.jpg,False,False,True,False
6664,ffff4eaa8.jpg,False,False,True,False


In [3]:
args = argparse.ArgumentParser(description='PyTorch Template')

config = ConfigParser(config=read_json(os.path.join(par_dir, 'config.json')))

data_loader = config.init_obj('data_loader', module_data)

train_loader = data_loader.train_loader
val_loader = data_loader.val_loader

model = config.init_obj('arch', module_arch)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\jupyter-notebook\\Kaggle\\Steel-Defect-Detection-Pytorch\\notebook\\config.json'

In [15]:
val_loader

In [ ]:
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str,
                    help='config file path (default: None)')
args.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')

# custom cli options to modify configuration from default values given in json file.
CustomArgs = collections.namedtuple('CustomArgs', 'flags type target')
options = [
    CustomArgs(['--lr', '--learning_rate'], type=float, target='optimizer;args;lr'),
    CustomArgs(['--bs', '--batch_size'], type=int, target='data_loader;args;batch_size')
]
config = ConfigParser.from_args(args, options)

logger = config.get_logger('train')

    # setup data_loader instances
    data_loader = config.init_obj('data_loader', module_data)
    train_loader = data_loader.train_loader
    val_loader = data_loader.val_loader

    # build model architecture, then print to console
    model = config.init_obj('arch', module_arch)
    logger.info(model)

    # prepare for (multi-device) GPU training
    device, device_ids = prepare_device(config['n_gpu'])
    model = model.to(device)
    if len(device_ids) > 1:
        model = torch.nn.DataParallel(model, device_ids=device_ids)

    # get function handles of loss and metrics
    criterion = getattr(module_loss, config['loss'])
    metrics = [getattr(module_metric, met) for met in config['metrics']]

    # build optimizer, learning rate scheduler. delete every lines containing lr_scheduler for disabling scheduler
    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = config.init_obj('optimizer', torch.optim, trainable_params)
    lr_scheduler = config.init_obj('lr_scheduler', torch.optim.lr_scheduler, optimizer)

    trainer = Trainer(model, criterion, metrics, optimizer,
                      config=config,
                      device=device,
                      train_loader=train_loader,
                      val_loader=val_loader,
                      lr_scheduler=lr_scheduler)

    trainer.train()